In [ ]:
from __future__ import division, print_function, absolute_import

from data import input_data
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import matplotlib.gridspec as gridspec

In [ ]:
# 생성된 이미지들을 저장할 generated_outputs 폴더를 생성합니다.
if not os.path.exists('./generated_output/GAN/'):
    os.makedirs('./generated_output/GAN/')

In [ ]:
# 생성된 MNIST 이미지를 8x8 Grid로 보여주는 plot 함수를 정의합니다.
def plot(samples):
    fig = plt.figure(figsize=(6, 6))
    gs = gridspec.GridSpec(6, 6)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        plt.imshow(sample.reshape(28, 28))
    return fig

In [ ]:
# 트레이닝 파라메터
learning_rate = 0.0002
num_steps = 30000
batch_size = 128

display_step = 2000

In [ ]:
# 네트워크 파라메터
gen_hidden_dim = 256 # generator hidden dim
disc_hidden_dim = 256 # discriminator hidden dim
num_input = 784 # MNIST data input (img shape: 28*28)
noise_dim = 100 # Noise data points

In [ ]:
# 변수 초기화 (see Xavier Glorot init)
def glorot_init(shape):
    return tf.random_normal(shape=shape, stddev=1. / tf.sqrt(shape[0] / 2.))

In [ ]:
# weight & bias 변수
weights = {
    'gen_hidden1': tf.Variable(glorot_init([noise_dim, gen_hidden_dim])),
    'gen_out': tf.Variable(glorot_init([gen_hidden_dim, num_input])),
    'disc_hidden1': tf.Variable(glorot_init([num_input, disc_hidden_dim])),
    'disc_out': tf.Variable(glorot_init([disc_hidden_dim, 1])),
}
biases = {
    'gen_hidden1': tf.Variable(tf.zeros([gen_hidden_dim])),
    'gen_out': tf.Variable(tf.zeros([num_input])),
    'disc_hidden1': tf.Variable(tf.zeros([disc_hidden_dim])),
    'disc_out': tf.Variable(tf.zeros([1])),
}

In [ ]:
# Generator
def generator(x):
    hidden_layer = tf.matmul(x, weights['gen_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['gen_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['gen_out'])
    out_layer = tf.add(out_layer, biases['gen_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

In [ ]:
# Discriminator
def discriminator(x):
    hidden_layer = tf.matmul(x, weights['disc_hidden1'])
    hidden_layer = tf.add(hidden_layer, biases['disc_hidden1'])
    hidden_layer = tf.nn.relu(hidden_layer)
    out_layer = tf.matmul(hidden_layer, weights['disc_out'])
    out_layer = tf.add(out_layer, biases['disc_out'])
    out_layer = tf.nn.sigmoid(out_layer)
    return out_layer

In [ ]:
# Build Networks
# Network Inputs
gen_input = tf.placeholder(tf.float32, shape=[None, noise_dim], name='input_noise')
disc_input = tf.placeholder(tf.float32, shape=[None, num_input], name='disc_input')

In [ ]:
# Generator Network
gen_sample = generator(gen_input)

# 두 개의 Discriminator Network 생성 (one from noise input, one from generated samples)
disc_real = discriminator(disc_input)
disc_fake = discriminator(gen_sample)

In [ ]:
# Loss Function
gen_loss = -tf.reduce_mean(tf.log(disc_fake))
disc_loss = -tf.reduce_mean(tf.log(disc_real) + tf.log(1. - disc_fake))

# Optimizer
optimizer_gen = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer_disc = tf.train.AdamOptimizer(learning_rate=learning_rate)


In [ ]:
# 각각의 optimizer 의 훈련 변수
# Generator Network 변수
gen_vars = [weights['gen_hidden1'], weights['gen_out'],
            biases['gen_hidden1'], biases['gen_out']]
# Discriminator Network Variables
disc_vars = [weights['disc_hidden1'], weights['disc_out'],
            biases['disc_hidden1'], biases['disc_out']]

# 최적화
train_gen = optimizer_gen.minimize(gen_loss, var_list=gen_vars)
train_disc = optimizer_disc.minimize(disc_loss, var_list=disc_vars)

# 변수 초기화
init = tf.global_variables_initializer()

In [ ]:
# TF session 시작
sess = tf.Session()

# initializer 실행
sess.run(init)

num_img = 0
# 훈련 시작
for i in range(1, num_steps+1):
    batch_x, _ = mnist.train.next_batch(batch_size)
    
    # Generator 에 feed 하기 위한 노이즈 생성
    z = np.random.uniform(-1., 1., size=[batch_size, noise_dim])
    
    # 500번 반복할때마다 생성된 이미지를 저장합니다.
    if i % 500 == 0:
        samples = sess.run(gen_sample, feed_dict={gen_input: np.random.uniform(-1., 1., [36, 100])})
        fig = plot(samples)
        plt.savefig('generated_output/GAN/%s-iter%d.png' % (str(num_img).zfill(3), i), bbox_inches='tight')
        num_img += 1
        plt.close(fig)
    
    # 훈련
    feed_dict = {disc_input: batch_x, gen_input: z}
    _, _, gl, dl = sess.run([train_gen, train_disc, gen_loss, disc_loss],
                            feed_dict=feed_dict)
    if i % display_step == 0 or i == 1:
        print('Step %i: Generator Loss: %f, Discriminator Loss: %f' % (i, gl, dl))


In [ ]:
plt.close()
# 테스트 시작
# 노이즈로부터 이미지를 생성
n = 6
canvas = np.empty((28 * n, 28 * n))
for i in range(n):
    # 노이즈 입력
    z = np.random.uniform(-1., 1., size=[n, noise_dim])
    
    # 이미지 생성
    g = sess.run(gen_sample, feed_dict={gen_input: z})
    
    g = -1 * (g - 1)
    for j in range(n):
        canvas[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

plt.figure(figsize=(n, n))
plt.imshow(canvas, origin="upper", cmap="gray")
plt.show()
plt.close()